# Tutorial 

**Note**: This guide assumes you have installed QIIME 2 using one of the procedures in the [install documents](https://docs.qiime2.org/2019.1/install/) and have installed [gemelli](https://github.com/biocore/gemelli).


## Introduction 

In this tutorial you will learn how to interpret and perform Phylogenetic Robust Aitchison PCA through QIIME 2. This tutorial builds from the Robust Aitchison PCA tutorial, which should be completed first.

In order to incorporate phylogenetic information we integrate the FastUniFrac method (see [here](https://www.nature.com/articles/ismej200997)) with the robust centered log ratio transform (rclr). The phylogenetic tree is denoted as $P\langle\Upsilon, \Lambda\rangle$ where the nodes of the tree are $\Upsilon=v_{1}, v_{2}, \ldots v_{a}$ and the branch weights $ A=\epsilon_{1}, \epsilon_{2}, \ldots \epsilon_{b}$. The total weight is defined as the sum of the branch lengths $W\left(\epsilon_{i}\right)$ which is vectorized and defined as $V_{l}$. In Fast UniFrac the distance between sample $x$ and $x^{\prime}$ is given as 

$$
d_{U}\left(x, x^{\prime}\right)=\frac{\sum_{i=1}^{m} V_{l} \cdot\left(x \oplus x^{\prime}\right)}{\sum_{i=1}^{m} V_{l} \cdot\left(x \vee x^{\prime}\right)}
$$

Updating the rclr in the previous tutorial to include the FastUniFrac transformation can be given as

$$
\begin{align}
 y_{aj }= \
\log( V_{l} \cdot x_{aj}) -\frac{1}{ \vert  \Omega _{V_{l} \cdot x_{a.}} \vert } \sum _{k \in  \Omega _{V_{l} \cdot x_{a.}}}^{}x_{k} - \frac{1}{ \vert  \Omega _{x_{.j}} \vert } \sum _{a \in  \Omega _{x_{.j}}}^{}x_{k} 
\end{align}
$$

The phylogenetic-RCLR can then be used with the Robust Aitchison PCA algorithm. To demonstrate this in action we will run an example dataset below, where the output can be viewed as a compositional biplot through emperor. 

## Example 


In this example we will use Phylogenetic Robust Aitchison PCA via gemelli on the “Moving Pictures” tutorial, if you have not yet completed the tutorial it can be found [here](https://docs.qiime2.org/2019.1/tutorials/moving-pictures/). The dataset consists of human microbiome samples from two individuals at four body sites at five timepoints, the first of which immediately followed antibiotic usage ([Caporaso et al. 2011](https://www.ncbi.nlm.nih.gov/pubmed/21624126)). If you have completed this tutorial run the following command and skip the download section.



##### Table [view](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Ftable.qza) | [download](https://docs.qiime2.org/2019.1/data/tutorials/moving-pictures/table.qza)
**save as:** table.qza 

##### Sample Metadata [download](https://data.qiime2.org/2019.1/tutorials/moving-pictures/sample_metadata.tsv)
**save as:** sample-metadata.tsv

##### Feature Metadata  [view](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Ftaxonomy.qza) | [download](https://docs.qiime2.org/2019.1/data/tutorials/moving-pictures/taxonomy.qza)
**save as:** taxonomy.qza

##### Rooted Phylogeny  [view](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2019.1%2Fdata%2Ftutorials%2Fmoving-pictures%2Frooted-tree.qza) | [download](https://docs.qiime2.org/2019.1/data/tutorials/moving-pictures/rooted-tree.qza)
**save as:** rooted-tree.qza


In [1]:
!cd qiime2-moving-pictures-tutorial

Using table.qza, of the type raw count table (FeatureTable[Frequency]), and rooted-tree.qza of type Phylogeny[Rooted], we will generate our beta diversity ordination file. The first tutorial covers many of the  parameters included in gemelli that we may want to consider. Phylogenetic RPCA included one additional parameter called `--p-min-depth`. This parameter will remove internal nodes of the tree that have less than the input number of descendants. This can be a useful method to prune large trees with high redundancy. 

**Note:** it is _not_ recommended to bin your features by taxonomic assignment (i.e. by genus level). 
**Note:** it is _not_ recommended to rarefy your data before using gemelli. 

Now that we understand the acceptable parameters, we are ready to run gemelli.  


In [2]:
!qiime dev refresh-cache

QIIME is caching your current deployment for improved performance. This may take a few moments and should only happen once per deployment.


In [8]:
!qiime gemelli phylogenetic-rpca-with-taxonomy \
    --i-table qiime2-moving-pictures-tutorial/table.qza \
    --i-phylogeny qiime2-moving-pictures-tutorial/rooted-tree.qza \
    --m-taxonomy-file qiime2-moving-pictures-tutorial/taxonomy.qza \
    --p-min-feature-count 10 \
    --p-min-sample-count 500 \
    --o-biplot qiime2-moving-pictures-tutorial/phylo-ordination.qza \
    --o-distance-matrix qiime2-moving-pictures-tutorial/phylo-distance.qza \
    --o-counts-by-node-tree qiime2-moving-pictures-tutorial/phylo-tree.qza \
    --o-counts-by-node qiime2-moving-pictures-tutorial/phylo-table.qza \
    --o-t2t-taxonomy qiime2-moving-pictures-tutorial/phylo-taxonomy.qza

Saved PCoAResults % Properties('biplot') to: qiime2-moving-pictures-tutorial/phylo-ordination.qza
Saved DistanceMatrix to: qiime2-moving-pictures-tutorial/phylo-distance.qza
Saved Phylogeny[Rooted] to: qiime2-moving-pictures-tutorial/phylo-tree.qza
Saved FeatureTable[Frequency] to: qiime2-moving-pictures-tutorial/phylo-table.qza
Saved FeatureData[Taxonomy] to: qiime2-moving-pictures-tutorial/phylo-taxonomy.qza


In addition to the ordination and distance that RPCA produces, phylogenetic RPCA produces a tree that was used, the table expanded to include the internal nodes, and a taxonomy file with lowest common ancestor taxonomy filled. 

Now that we have our ordination file, with type (PCoAResults % Properties(['biplot'])), we are ready to visualize the results. In this tutorial we can view both the biplot and the phylogeny at the same time with [empress](https://github.com/biocore/empress). In this case we will include metadata for our features (optional) and our samples (required). 



In [11]:
!qiime empress community-plot\
    --i-tree qiime2-moving-pictures-tutorial/phylo-tree.qza\
    --i-feature-table qiime2-moving-pictures-tutorial/phylo-table.qza\
    --i-pcoa qiime2-moving-pictures-tutorial/phylo-ordination.qza\
    --m-sample-metadata-file qiime2-moving-pictures-tutorial/sample-metadata.tsv\
    --m-feature-metadata-file qiime2-moving-pictures-tutorial/phylo-taxonomy.qza\
    --p-filter-missing-features\
    --p-number-of-features 50\
    --o-visualization qiime2-moving-pictures-tutorial/phylo-empress.qzv


Saved Visualization to: qiime2-moving-pictures-tutorial/phylo-empress.qzv


Biplots are exploratory visualization tools that allow us to represent the features (i.e. taxonomy, OTUs, or internal nodes of the phylogeny)  that strongly influence the principal component axis as arrows. The interpretation of the compositional biplot differs slightly from classical biplot interpretation. The previous tutorial describes how to interpret the biplot in more depth.The important features with regard to sample clusters are not a single arrow but by the log ratio between features represented by arrows pointing in different directions.  To effectively use Emperor we fist will color the samples by BodySite. 

Furthermore, unlike the RPCA output, Phylo-RPCA includes internal nodes in the arrows with taxonomy labeled by LCA. Following the similar methodology in the previous tutorial we can observed that BodySite seems to explain the clusters well. 

![](etc/img21.png)


From this visualization we noticed that BodySite seems to explain the clusters well. We can run [PERMANOVA](https://docs.qiime2.org/2019.1/plugins/available/diversity/beta-group-significance/) on the distances to get a statistical significance for this. 

In [9]:
!qiime diversity beta-group-significance \
    --i-distance-matrix qiime2-moving-pictures-tutorial/phylo-distance.qza \
    --m-metadata-file qiime2-moving-pictures-tutorial/sample-metadata.tsv \
    --m-metadata-column BodySite \
    --p-method permanova \
    --o-visualization qiime2-moving-pictures-tutorial/phylo-BodySite_significance.qzv

Saved Visualization to: qiime2-moving-pictures-tutorial/phylo-BodySite_significance.qzv


Indeed we can now see that the clusters we saw in the biplot were significant by viewing the phylo-BodySite_significance.qzv at [view.qiime2](https://view.qiime2.org).

![](etc/img19.png)

We can see from the biplot and PERMANOVA results that gut is very different from the skin samples. 

Next we can color the internal nodes and sOTUs by their LCA and directly predicted taxonomy respectively. 

![](etc/img22.png)

By double clicking on the arrows associated with the different body sites we can explore where they fall in the phylogeny. For example Faecalibacterium (g) associated with gut

![](etc/img23.png)


and Streptococcus (g) associated with the skin/tongue body site samples.

![](etc/img24.png)


Next we can use [qurro](https://github.com/biocore/qurro) to explore log-ratios of the phylogenetic partitions of microbes highlighted by gemelli. For more about why log-ratios are useful you may want to read ["Establishing microbial composition measurement standards with reference frames"](https://www.nature.com/articles/s41467-019-10656-5).

**Note: It is important that log-ratios are not taken between nodes that shared descendants**


In [1]:
!qiime qurro loading-plot \
    --i-ranks qiime2-moving-pictures-tutorial/phylo-ordination.qza \
    --i-table qiime2-moving-pictures-tutorial/phylo-table.qza \
    --m-sample-metadata-file qiime2-moving-pictures-tutorial/sample-metadata.tsv \
    --m-feature-metadata-file qiime2-moving-pictures-tutorial/phylo-taxonomy.qza \
    --o-visualization qiime2-moving-pictures-tutorial/phylo-qurro_plot.qzv


Saved Visualization to: qiime2-moving-pictures-tutorial/phylo-qurro_plot.qzv


Two taxa groups whose arrows seem to be directly opposed with relation to the BodySite grouping is Faecalibacterium (associated with gut) and Streptococcus (associated with skin and oral). We can use Qurro to explore this relationship. To make a log-ratio we can filter by taxa who contain Faecalibacterium in the numerator and Streptococcus in the denominator of the log-ratio. Those features will then be summed according to thier taxonomic labels and used in the log-ratio. In Qurro the axis one loadings (or another axis) from gemelli are highlighted by if they are contained in the numerator or denominator. The log-ratio plot is contained on the left and can be visualized as a scatter or box-plot. From this it is clear these taxa can separate our BodySite groupings. The tsv file can be exported and a t-test by BodySite on the log-ratos could confirm this observation.

![](etc/img26.png)

